In [2]:
from processing import *
# from generate_dataset import *
from clf import get_clf
from PIL import Image

import warnings  #suppress warnings
warnings.simplefilter('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)


from sklearn import svm
from matplotlib.colors import Normalize

from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV

from tensorflow.keras.datasets import mnist
(X_tr,y_tr),(X_ts,y_ts) = mnist.load_data()


In [105]:
results = pd.read_csv('subset.csv',header=None)
results.columns = ['image','truth']
results

,image,truth
0,generated_imageset/merge_4_82____2659.png,"[2,6,5,9]"
1,generated_imageset/merge_4_277____1825.png,"[1,8,2,5]"
2,generated_imageset/merge_4_313____2726.png,"[2,7,2,6]"
3,generated_imageset/merge_4_414____8426.png,"[8,4,2,6]"
4,generated_imageset/merge_4_568____8427.png,"[8,4,2,7]"
...,...,...
171,generated_imageset/merge_4_17263____6482.png,"[6,4,8,2]"
172,generated_imageset/merge_4_17352____7450.png,"[7,4,5,0]"
173,generated_imageset/merge_4_17386____4035.png,"[4,0,3,5]"
174,generated_imageset/merge_4_17394____0178.png,"[0,1,7,8]"


In [3]:
X_tr_flat = [x.flatten() for x in X_tr][:10000]
X_ts_flat = [x.flatten() for x in X_ts]

scaler = StandardScaler()
X_tr_scaled = scaler.fit_transform(X_tr_flat)
y_tr_scaled = y_tr[:10000]

In [4]:
svm_model = svm.SVC()
svm_model.fit(X_tr_scaled,y_tr_scaled)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [5]:
import pickle
from datetime import datetime
saved_model_path = "./saved_models/baseline_SVM_{}.p".format(datetime.now().strftime("%Y%m%d%s")) # _%H%M%S
# Save entire model to a HDF5 file
# fitted_clf.save(saved_model_path)
pickle.dump(svm_model, open(saved_model_path, 'wb'))

In [34]:
directory = 'INKdata/MatricesTrain2014/'
out_folder= './MatricesTrain_png/'
png_files = []
f = os.popen("ls %s" %out_folder)
for i in f.readlines():
    png_files.append(out_folder + i[:-1])
f.close()    

In [39]:
# binary,label_arr,segments,image=label_segments('./MatricesTrain_png/104_em_99.png','SVC/output__')
# # crop_image(segments[1],label_arr,image,model=svm_model,direc='tempimgs/',svc=True)
# binary.shape
# plt.imshow(binary)
# for segment in segments:
#     found = label_arr == segment    
#     #plt.imshow(found)
#     x,y = np.where(found)
#     xmin,xmax,ymin,ymax = np.min(x),np.max(x),np.min(y),np.max(y)
#     xlen,ylen = found[xmin:xmax,ymin:ymax].shape
#     diff = np.abs(ylen-xlen)
#     change = ceil(diff/2)
#     if diff!=0:
#         if ylen>xlen:
#             xmin-=change
#             xmax+=change

#         else:
#             ymin-=change
#             ymax+=change
#         xlen,ylen = xmax-xmin,ymax-ymin
#         diff=np.abs(ylen-xlen)
#         if xlen>ylen: ymax+=diff
#         elif ylen>xlen: xmax+=diff

#     prediction=None
#     digit = np.invert(binary[xmin:xmax,ymin:ymax])
#     digit = np.pad(digit,int(len(digit)*.2))
#     xlen,ylen = digit.shape
#     print(digit.shape)
png_files=['images/matrix24_original.png']
predictions = []
for index,image in enumerate(png_files[0]):
    predictions.append(process_image(image,'SVC/output__',plot=True,fitted_clf=svm_model,svc=True))
predictions=np.array(predictions)
predictions

FileNotFoundError: No such file: '/home/nina/autograder/i'

In [8]:
predictions = []
for index,image in enumerate(results['image']):
    predictions.append(process_image(image,'SVC/output__',fitted_clf=svm_model,svc=True))
predictions=np.array(predictions)
results['predict'] = predictions

Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.


ValueError: too many values to unpack (expected 2)

In [109]:
def setdiff(x):
    return x['truth']==x['predict']
results['correct']=results.apply(setdiff,axis=1)
results

,image,truth,predict,correct
0,generated_imageset/merge_4_82____2659.png,"[2,6,5,9]","[3, 8, None, None, 8]","[False, False, False, False, False]"
1,generated_imageset/merge_4_277____1825.png,"[1,8,2,5]","[8, 8, 2, 2]",False
2,generated_imageset/merge_4_313____2726.png,"[2,7,2,6]","[2, 2, 3, 8]",False
3,generated_imageset/merge_4_414____8426.png,"[8,4,2,6]","[8, 4, 2, 0]",False
4,generated_imageset/merge_4_568____8427.png,"[8,4,2,7]","[8, 4, 2, 8]",False
...,...,...,...,...
171,generated_imageset/merge_4_17263____6482.png,"[6,4,8,2]","[6, None, None, 8, 2]","[False, False, False, False, False]"
172,generated_imageset/merge_4_17352____7450.png,"[7,4,5,0]","[8, 4, 5, 0]",False
173,generated_imageset/merge_4_17386____4035.png,"[4,0,3,5]","[4, 0, 3, 5]",False
174,generated_imageset/merge_4_17394____0178.png,"[0,1,7,8]","[2, 5, 8, 8]",False


In [113]:
results_SVM = main(svc=True,fitted_clf=svm_model)

Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert imag

Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert imag

Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert imag

Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert imag

Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert imag

...

Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert imag

Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert imag

Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert imag

Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert imag

Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert imag

In [115]:
results_SVM.to_csv('finalSVM_results.csv')

In [117]:
results_SVM['string']= results_SVM.correct.astype(str)

In [118]:
def getscore(x):
    if x['string']=='False':
        return -10
    else:
        return x['string'].count('True')/4

In [121]:
results_SVM['score'] = results_SVM.apply(getscore,axis=1)

In [122]:
mismatched = results_SVM[results_SVM.score<0]

In [124]:
mainset = results_SVM[results_SVM.score>=0]
mainset.drop('string',axis=1,inplace=True)

In [126]:
mismatched = results_SVM[results_SVM.score<0]
mismatched.drop('string',axis=1,inplace=True)

In [143]:
mismatched

,image,truth,predict,correct,score
7,generated_imageset/merge_4_7____3982.png,"[3, 9, 8, 2]","[3, None, 2, 8, 2]",False,-10.0
8,generated_imageset/merge_4_8____1551.png,"[1, 5, 5, 1]","[1, 2, 2, 8, 8]",False,-10.0
17,generated_imageset/merge_4_17____9373.png,"[9, 3, 7, 3]","[8, 3, 2, None, None]",False,-10.0
21,generated_imageset/merge_4_21____0245.png,"[0, 2, 4, 5]","[5, 2, 4, None, None, 2]",False,-10.0
25,generated_imageset/merge_4_25____1037.png,"[1, 0, 3, 7]","[8, 0, 3, 2, None]",False,-10.0
...,...,...,...,...,...
17477,generated_imageset/merge_4_17477____5949.png,"[5, 9, 4, 9]","[5, 8, None, None, 8]",False,-10.0
17479,generated_imageset/merge_4_17479____4067.png,"[4, 0, 6, 7]","[4, 0, 6, None, 8]",False,-10.0
17485,generated_imageset/merge_4_17485____5881.png,"[5, 8, 8, 1]","[None, 4, 8, 8, 1]",False,-10.0
17488,generated_imageset/merge_4_17488____3085.png,"[3, 0, 8, 5]","[3, 0, 8, None, 4]",False,-10.0


In [136]:
mainset.tail(30)

,image,truth,predict,correct,score
17460,generated_imageset/merge_4_17460____0414.png,"[0, 4, 1, 4]","[0, 4, 1, 4]","[True, True, True, True]",1.00
17461,generated_imageset/merge_4_17461____8443.png,"[8, 4, 4, 3]","[8, 4, 4, 3]","[True, True, True, True]",1.00
17463,generated_imageset/merge_4_17463____2446.png,"[2, 4, 4, 6]","[2, 4, 4, 8]","[True, True, True, False]",0.75
17464,generated_imageset/merge_4_17464____7761.png,"[7, 7, 6, 1]","[2, 2, 6, 8]","[False, False, True, False]",0.25
17465,generated_imageset/merge_4_17465____4866.png,"[4, 8, 6, 6]","[4, 8, 6, 8]","[True, True, True, False]",0.75
17466,generated_imageset/merge_4_17466____1170.png,"[1, 1, 7, 0]","[1, 1, 2, 0]","[True, True, False, True]",0.75
17468,generated_imageset/merge_4_17468____0351.png,"[0, 3, 5, 1]","[0, 2, 5, 1]","[True, False, True, True]",0.75
17470,generated_imageset/merge_4_17470____9106.png,"[9, 1, 0, 6]","[3, 8, 0, 0]","[False, False, True, False]",0.25
17473,generated_imageset/merge_4_17473____6187.png,"[6, 1, 8, 7]","[0, 7, 8, 5]","[False, False, True, False]",0.25
17474,generated_imageset/merge_4_17474____9427.png,"[9, 4, 2, 7]","[8, 4, 2, 2]","[False, True, True, False]",0.50


In [129]:
mainset.describe()

,score
count,15042.000000
mean,0.579095
std,0.246942
min,0.000000
25%,0.500000
50%,0.500000
75%,0.750000
max,1.000000


In [ ]:
def getscore(x):
    if x['string']=='False':
        return -10
    else:
        return x['string'].count('True')/4
def process_results(results4):
    results4['string']= results4.correct.astype(str)
    results4['score'] = results4.apply(getscore,axis=1)    
    mismatched = results4[results4.score<0]
    mainset = results4[results4.score>=0]
    mismatched.drop('string',axis=1,inplace=True)
    mainset.drop('string',axis=1,inplace=True)
    return results4, mismatched, mainset

In [65]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})".format(results['mean_test_score'][candidate],results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))

In [70]:
##tuning svm
C_range = np.logspace(2, 10, 2)
gamma_range = np.logspace(-9, 3, 2)
param_grid = dict(gamma=gamma_range, C=C_range)
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=42)
grid = GridSearchCV(SVC(), param_grid=param_grid, cv=cv)
grid.fit(X_tr_scaled,y_tr_scaled)
print("The best parameters are %s with a score of %0.2f "   % (grid.best_params_, grid.best_score_))
print(report(grid.cv_results_))


# ####The best parameters are {'C': 10000000000.0, 'gamma': 1e-09} with a score of 0.90 
# Model with rank: 1
# Mean validation score: 0.904 (std: 0.009)
# Parameters: {'C': 10000000000.0, 'gamma': 1e-09}
# Model with rank: 2
# Mean validation score: 0.113 (std: 0.000)
# Parameters: {'C': 100.0, 'gamma': 1e-09}
# Model with rank: 2
# Mean validation score: 0.113 (std: 0.000)
# Parameters: {'C': 100.0, 'gamma': 1000.0}
# Model with rank: 2
# Mean validation score: 0.113 (std: 0.000)
# Parameters: {'C': 10000000000.0, 'gamma': 1000.0}
# None


The best parameters are {'C': 10000000000.0, 'gamma': 1e-09} with a score of 0.90 
Model with rank: 1
Mean validation score: 0.904 (std: 0.009)
Parameters: {'C': 10000000000.0, 'gamma': 1e-09}
Model with rank: 2
Mean validation score: 0.113 (std: 0.000)
Parameters: {'C': 100.0, 'gamma': 1e-09}
Model with rank: 2
Mean validation score: 0.113 (std: 0.000)
Parameters: {'C': 100.0, 'gamma': 1000.0}
Model with rank: 2
Mean validation score: 0.113 (std: 0.000)
Parameters: {'C': 10000000000.0, 'gamma': 1000.0}
None


In [62]:
friday = make_stuff(size=20000)

In [63]:
SVC_results = main(friday,svc=True,fitted_clf=svm_model)

Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert imag

Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert imag

Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert imag

Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert imag

Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert imag

...

Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert imag

Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert imag

Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert imag

Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert imag

Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert image to uint8 prior to saving to suppress this warning.
Lossy conversion from int64 to uint8. Range [0, 1]. Convert imag

In [66]:
results1000, mismatched1000, mainset1000 = process_results(SVC_results)

In [67]:
results1000

,image,truth,predict,correct,string,score
0,generated_imageset/merge_4_0____1810.png,"[1, 8, 1, 0]","[8, 8, 8, 5]","[False, True, False, False]",[False True False False],0.25
1,generated_imageset/merge_4_1____6072.png,"[6, 0, 7, 2]","[6, 0, 3, 2]","[True, True, False, True]",[ True True False True],0.75
2,generated_imageset/merge_4_2____6675.png,"[6, 6, 7, 5]","[2, 6, 2, 5]","[False, True, False, True]",[False True False True],0.50
3,generated_imageset/merge_4_3____9865.png,"[9, 8, 6, 5]","[8, 8, 0, 2]","[False, True, False, False]",[False True False False],0.25
4,generated_imageset/merge_4_4____8272.png,"[8, 2, 7, 2]","[8, 2, 2, 2]","[True, True, False, True]",[ True True False True],0.75
...,...,...,...,...,...,...
4995,generated_imageset/merge_4_4995____2108.png,"[2, 1, 0, 8]","[2, 8, 0, 8]","[True, False, True, True]",[ True False True True],0.75
4996,generated_imageset/merge_4_4996____1104.png,"[1, 1, 0, 4]","[8, 1, 0, 4]","[False, True, True, True]",[False True True True],0.75
4997,generated_imageset/merge_4_4997____3451.png,"[3, 4, 5, 1]","[3, 4, 5, 8]","[True, True, True, False]",[ True True True False],0.75
4998,generated_imageset/merge_4_4998____1290.png,"[1, 2, 9, 0]","[8, 2, 8, 0]","[False, True, False, True]",[False True False True],0.50


In [68]:
mismatched1000.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 723 entries, 9 to 4984
Data columns (total 5 columns):
image      723 non-null object
truth      723 non-null object
predict    723 non-null object
correct    723 non-null object
score      723 non-null float64
dtypes: float64(1), object(4)
memory usage: 33.9+ KB


In [69]:
mainset1000.describe()

,score
count,4277.000000
mean,0.586801
std,0.245618
min,0.000000
25%,0.500000
50%,0.500000
75%,0.750000
max,1.000000
